In [11]:
import os
from pathlib import Path
import yaml
import cv2
import random

# -------------------------------------------------
# CHANGE THIS to your yaml location
YAML_PATH = r"C:\Final project 2\backend\datasets\data.yaml"
# -------------------------------------------------

# ---- helper: load yaml ----
with open(YAML_PATH, "r") as f:
    data_cfg = yaml.safe_load(f)

base_dir = Path(YAML_PATH).parent
train_imgs_dir = (base_dir / data_cfg["train"]).resolve()
train_lbls_dir = (base_dir / data_cfg["train"].replace("images", "labels")).resolve()

nc = data_cfg["nc"]
class_names = data_cfg["names"]
print("Found classes:", class_names)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

def enhance_image(img):
    # CLAHE in LAB + light unsharp
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    img_clahe = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    gaussian = cv2.GaussianBlur(img_clahe, (0, 0), 3)
    sharpened = cv2.addWeighted(img_clahe, 1.4, gaussian, -0.4, 0)
    return sharpened

def load_label(path: Path):
    if not path.exists():
        return []
    anns = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            cls_id = int(parts[0])
            xywh = list(map(float, parts[1:]))
            anns.append((cls_id, xywh))
    return anns

def save_label(path: Path, anns):
    with open(path, "w") as f:
        for cls_id, xywh in anns:
            xywh_str = " ".join(f"{v:.6f}" for v in xywh)
            f.write(f"{cls_id} {xywh_str}\n")

def flip_h(img, anns):
    # horizontal flip: x -> 1 - x
    flipped = cv2.flip(img, 1)
    new_anns = []
    for cls_id, (x, y, w, h) in anns:
        new_anns.append((cls_id, (1.0 - x, y, w, h)))
    return flipped, new_anns

def flip_v(img, anns):
    # vertical flip: y -> 1 - y
    flipped = cv2.flip(img, 0)
    new_anns = []
    for cls_id, (x, y, w, h) in anns:
        new_anns.append((cls_id, (x, 1.0 - y, w, h)))
    return flipped, new_anns

def adjust_brightness(img, factor=1.2):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.multiply(v, factor)
    v = cv2.min(v, 255).astype("uint8")
    hsv = cv2.merge([h, s, v])
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# 1) scan all train images and build: class_id -> list of (img_path, lbl_path)
per_class_images = {i: [] for i in range(nc)}

all_img_paths = [p for p in train_imgs_dir.iterdir() if p.suffix.lower() in IMG_EXTS]
print(f"Train images found: {len(all_img_paths)}")

for img_path in all_img_paths:
    lbl_path = train_lbls_dir / f"{img_path.stem}.txt"
    anns = load_label(lbl_path)
    present_classes = {cls_id for cls_id, _ in anns}
    for cls_id in present_classes:
        if cls_id in per_class_images:
            per_class_images[cls_id].append((img_path, lbl_path))

# 2) count per class
counts = {cls_id: len(imgs) for cls_id, imgs in per_class_images.items()}
print("Counts per class:")
for cls_id, cnt in counts.items():
    print(f"- {cls_id} ({class_names[cls_id]}): {cnt}")

max_count = max(counts.values())
print("Max count =", max_count)

# 3) for every class with < max_count, augment until it matches
for cls_id in range(nc):
    cls_name = class_names[cls_id]
    current_list = per_class_images[cls_id]
    current_count = len(current_list)

    if current_count == 0:
        print(f"⚠️ Class {cls_id} ({cls_name}) has 0 images, skipping.")
        continue

    need = max_count - current_count
    if need <= 0:
        print(f"Class {cls_id} ({cls_name}) is already at max ({current_count}).")
        continue

    print(f"Augmenting class {cls_id} ({cls_name}) to add {need} images...")
    i = 0
    while need > 0:
        src_img_path, src_lbl_path = current_list[i % len(current_list)]
        img = cv2.imread(str(src_img_path))
        anns = load_label(src_lbl_path)

        # always enhance first
        img = enhance_image(img)

        aug_choice = random.choice(["hflip", "vflip", "bright"])

        if aug_choice == "hflip":
            aug_img, aug_anns = flip_h(img, anns)
            suffix = "hflip"
        elif aug_choice == "vflip":
            aug_img, aug_anns = flip_v(img, anns)
            suffix = "vflip"
        else:
            aug_img = adjust_brightness(img, factor=random.uniform(1.1, 1.4))
            aug_anns = anns
            suffix = "bright"

        new_name = f"{src_img_path.stem}_aug_{class_names[cls_id].replace(' ', '')}_{need}_{suffix}.jpg"
        new_img_path = train_imgs_dir / new_name
        new_lbl_path = train_lbls_dir / f"{new_img_path.stem}.txt"

        cv2.imwrite(str(new_img_path), aug_img)
        save_label(new_lbl_path, aug_anns)

        need -= 1
        i += 1

    print(f"✅ Done augmenting class {cls_id} ({cls_name}).")

print("✅ Dataset balancing + enhancement + your 2 main augmentations (flip, brightness) done.")


Found classes: ['Comminuted', 'Greenstick', 'Healthy', 'Linear', 'Oblique Displaced', 'Oblique', 'Segmental', 'Spiral', 'Transverse Displaced', 'Transverse']
Train images found: 1347
Counts per class:
- 0 (Comminuted): 156
- 1 (Greenstick): 81
- 2 (Healthy): 54
- 3 (Linear): 21
- 4 (Oblique Displaced): 336
- 5 (Oblique): 48
- 6 (Segmental): 18
- 7 (Spiral): 66
- 8 (Transverse Displaced): 564
- 9 (Transverse): 105
Max count = 564
Augmenting class 0 (Comminuted) to add 408 images...
✅ Done augmenting class 0 (Comminuted).
Augmenting class 1 (Greenstick) to add 483 images...
✅ Done augmenting class 1 (Greenstick).
Augmenting class 2 (Healthy) to add 510 images...
✅ Done augmenting class 2 (Healthy).
Augmenting class 3 (Linear) to add 543 images...
✅ Done augmenting class 3 (Linear).
Augmenting class 4 (Oblique Displaced) to add 228 images...
✅ Done augmenting class 4 (Oblique Displaced).
Augmenting class 5 (Oblique) to add 516 images...
✅ Done augmenting class 5 (Oblique).
Augmenting clas

In [12]:
from ultralytics import YOLO

model = YOLO("yolo11s.pt")  # or yolo11s.pt if you have more VRAM

model.train(
    data=r"C:\Final project 2\backend\datasets\data.yaml",  # your YAML
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,          # 0 = your GPU, or "cpu"
    workers=4,
    # match your requested augmentations inside the loader too:
    fliplr=0.5,        # horizontal flip prob
    flipud=0.3,        # vertical flip prob
    hsv_v=0.4,         # brightness/value jitter
    patience=20,
)


New https://pypi.org/project/ultralytics/8.3.226 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Final project 2\backend\datasets\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.3, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, nam

c:\Users\aadit\anaconda3\envs\fxdetector\lib\site-packages\albumentations\core\validation.py:34: UserWarning: Argument 'quality_range' is not valid and will be ignored.
  warn(


val: Fast image access  (ping: 0.00.0 ms, read: 384.183.1 MB/s, size: 25.9 KB)
val: Scanning C:\Final project 2\backend\datasets\valid\labels... 128 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 2.4Kit/s 0.1s
val: New cache created: C:\Final project 2\backend\datasets\valid\labels.cache
Plotting labels to C:\Final project 2\backend\notebooks\runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to C:\Final project 2\backend\notebooks\runs\detect\train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      4.15G      1.957      3.954      1.971          4        640: 10

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EBBB67E290>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

In [18]:
from ultralytics import YOLO

# 1) load your trained weights
model = YOLO(r"C:\Final project 2\backend\notebooks\runs\detect\train\weights\best.pt")   # <-- change to your actual path

# 2) evaluate on your validation set
metrics = model.val(data=r"C:\Final project 2\backend\datasets\data.yaml", imgsz=640)

print("mAP50:", metrics.box.map50)

# this is mAP@0.5:0.95 (the COCO-style one)
print("mAP50-95:", metrics.box.map)

# mean precision / recall over classes
print("Mean precision:", metrics.box.mp)
print("Mean recall:", metrics.box.mr)

# per-class mAP50 (length = 10 for you)
print("Per-class mAP50:")
for i, v in enumerate(metrics.box.maps):
    print(i, v)

Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
YOLO11s summary (fused): 100 layers, 9,416,670 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 589.6163.2 MB/s, size: 31.9 KB)
val: Scanning C:\Final project 2\backend\datasets\valid\labels.cache... 128 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 3.3it/s 2.4s0.2ss
                   all        128        157      0.956      0.868      0.919       0.51
            Comminuted         15         15          1      0.775      0.832      0.494
            Greenstick         10         10      0.967        0.6      0.718      0.295
               Healthy          7          7      0.957          1      0.995       0.81
                Linear          1          1      0.819          1      0.995      0.298
     Oblique